# Hand-written Decision Tree

## Achievement:
* 重复了周志华《机器学习》（西瓜书）的 Fig 4.5-4.8
* ID3, CART, C4.5
* pre-pruning, post-pruning
* Pretty_print and mermaid representation of the tree

## To-do list: 
* 对付缺失值



## Initialization

In [1]:

from collections import Counter
from time import time
import numpy as np
import pandas as pd
import re

# initialization
dataSet = [
        # 1
        ['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', 0.697, 0.460, '好瓜'],
        # 2
        ['乌黑', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', 0.774, 0.376, '好瓜'],
        # 3
        ['乌黑', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', 0.634, 0.264, '好瓜'],
        # 4
        ['青绿', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', 0.608, 0.318, '好瓜'],
        # 5
        ['浅白', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', 0.556, 0.215, '好瓜'],
        # 6
        ['青绿', '稍蜷', '浊响', '清晰', '稍凹', '软粘', 0.403, 0.237, '好瓜'],
        # 7
        ['乌黑', '稍蜷', '浊响', '稍糊', '稍凹', '软粘', 0.481, 0.149, '好瓜'],
        # 8
        ['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '硬滑', 0.437, 0.211, '好瓜'],

        # ----------------------------------------------------
        # 9
        ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '硬滑', 0.666, 0.091, '坏瓜'],
        # 10
        ['青绿', '硬挺', '清脆', '清晰', '平坦', '软粘', 0.243, 0.267, '坏瓜'],
        # 11
        ['浅白', '硬挺', '清脆', '模糊', '平坦', '硬滑', 0.245, 0.057, '坏瓜'],
        # 12
        ['浅白', '蜷缩', '浊响', '模糊', '平坦', '软粘', 0.343, 0.099, '坏瓜'],
        # 13
        ['青绿', '稍蜷', '浊响', '稍糊', '凹陷', '硬滑', 0.639, 0.161, '坏瓜'],
        # 14
        ['浅白', '稍蜷', '沉闷', '稍糊', '凹陷', '硬滑', 0.657, 0.198, '坏瓜'],
        # 15
        ['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '软粘', 0.360, 0.370, '坏瓜'],
        # 16
        ['浅白', '蜷缩', '浊响', '模糊', '平坦', '硬滑', 0.593, 0.042, '坏瓜'],
        # 17
        ['青绿', '蜷缩', '沉闷', '稍糊', '稍凹', '硬滑', 0.719, 0.103, '坏瓜']
    ]

    # 特征值列表
label = ['色泽', '根蒂', '敲击', '纹理', '脐部', '触感', '密度', '含糖率', 'Good']
is_continuous = [False]*6+[True]*2

      

## dataset processing

In [2]:
dataSet = pd.DataFrame(dataSet,columns=label)
d_list =  [{'乌黑':1,'青绿':0,'浅白':-1}, 
           {'蜷缩':1,'稍蜷':0,'硬挺':-1}, 
           {'沉闷':1,'浊响':0,'清脆':-1}, 
           {'清晰':1,'稍糊':0,'模糊':-1},
           {'凹陷':1,'稍凹':0,'平坦':-1},
           {'硬滑':1,'软粘':-1},
           {'≤':-1,'>':1},
           {'≤':-1,'>':1},
           {'好瓜':1,'坏瓜':-1}]
dataSet['色泽'] = dataSet['色泽'].map({'乌黑':1,'青绿':0,'浅白':-1})
dataSet['根蒂'] = dataSet['根蒂'].map({'蜷缩':1,'稍蜷':0,'硬挺':-1})
dataSet['敲击'] = dataSet['敲击'].map({'沉闷':1,'浊响':0,'清脆':-1})
dataSet['纹理'] = dataSet['纹理'].map({'清晰':1,'稍糊':0,'模糊':-1})
dataSet['脐部'] = dataSet['脐部'].map({'凹陷':1,'稍凹':0,'平坦':-1})
dataSet['触感'] = dataSet['触感'].map({'硬滑':1,'软粘':-1})
dataSet['Good'] = dataSet['Good'].map({'好瓜':1,'坏瓜':-1})

# print(dataSet)

rev_d_list = [{value:key for key,value in d.items()} for d in d_list]


    色泽  根蒂  敲击  纹理  脐部  触感     密度    含糖率  Good
0    0   1   0   1   1   1  0.697  0.460     1
1    1   1   1   1   1   1  0.774  0.376     1
2    1   1   0   1   1   1  0.634  0.264     1
3    0   1   1   1   1   1  0.608  0.318     1
4   -1   1   0   1   1   1  0.556  0.215     1
5    0   0   0   1   0  -1  0.403  0.237     1
6    1   0   0   0   0  -1  0.481  0.149     1
7    1   0   0   1   0   1  0.437  0.211     1
8    1   0   1   0   0   1  0.666  0.091    -1
9    0  -1  -1   1  -1  -1  0.243  0.267    -1
10  -1  -1  -1  -1  -1   1  0.245  0.057    -1
11  -1   1   0  -1  -1  -1  0.343  0.099    -1
12   0   0   0   0   1   1  0.639  0.161    -1
13  -1   0   1   0   1   1  0.657  0.198    -1
14   1   0   0   1   0  -1  0.360  0.370    -1
15  -1   1   0  -1  -1   1  0.593  0.042    -1
16   0   1   1   0   0   1  0.719  0.103    -1


## Define functions and classes

In [16]:
def ent(y): # OK
    p = np.array([np.count_nonzero(y==i)/y.size for i in np.unique(y)])
    return -p@np.log2(p)

def gain(X, y, a, continuous): # OK
    """Possible issue: attributes.size <= 1?
    """
    attributes = np.unique(X[:,a])
    if not continuous:
        y_new = [ y[X[:,a]==i] for i in attributes]
        result = ent(y) - sum(i.size*ent(i) for i in y_new)/y.size
        return (result, None)
    else:
        if attributes.size <= 1:
            return (-1, None) # should never be chosen
        result = 0
        split_point = (attributes[0]+attributes[1])/2
        for i in range(attributes.size-1):
            current_split_point = (attributes[i]+attributes[i+1])/2
            y1 = y[X[:,a]<=current_split_point]
            y2 = y[X[:,a]>current_split_point]
            current_gain = ent(y)-(y1.size*ent(y1)+y2.size*ent(y2))/y.size
            if current_gain > result:
                result = current_gain
                split_point = current_split_point
        return (result, split_point)
        
    
def gain_ratio(X, y, a, continuous): # OK
    attributes = np.unique(X[:,a])
    if not continuous:
        if attributes.size == 1:
            return (0, None)
        y_new = [ y[X[:,a]==i] for i in attributes]
        gain = ent(y) - sum(i.size*ent(i) for i in y_new)/y.size
        intrinsic_value = - sum(i.size*np.log2(i.size/y.size) for i in y_new)/y.size
        return (gain/intrinsic_value, None)
    else:
        if attributes.size <= 1:
            return (-1, None) # should never be chosen
        result = 0
        split_point = (attributes[0]+attributes[1])/2
        for i in range(attributes.size-1):
            current_split_point = (attributes[i]+attributes[i+1])/2
            y1 = y[X[:,a]<=current_split_point]
            y2 = y[X[:,a]>current_split_point]
            gain = ent(y)-(y1.size*ent(y1)+y2.size*ent(y2))/y.size
            intrinsic_value = -(y1.size*np.log2(y1.size/y.size)+y2.size*np.log2(y2.size/y.size))/y.size
            current_gain_ratio = gain/intrinsic_value
            if current_gain_ratio > result:
                result = current_gain_ratio
                split_point = current_split_point
        return (result, split_point)
        
    
def gini(y): # OK
    p = np.array([np.count_nonzero(y==i)/y.size for i in np.unique(y)])
    return 1-p@p

def gini_index(X, y, a, continuous): # OK
    attributes = np.unique(X[:,a])
    if not continuous:
        y_new = [ y[X[:,a]==i] for i in attributes]
        return (sum(i.size*gini(i) for i in y_new)/y.size, None)
    else:
        if attributes.size <= 1:
            return (2, None) # should never be chosen
        result = 1
        split_point = (attributes[0]+attributes[1])/2
        for i in range(attributes.size-1):
            current_split_point = (attributes[i]+attributes[i+1])/2
            y1 = y[X[:,a]<=current_split_point]
            y2 = y[X[:,a]>current_split_point]
            current_gini_index = y1.size/y.size*gini(y1)+y2.size/y.size*gini(y2)
            if current_gini_index < result:
                result = current_gini_index
                split_point = current_split_point
        return (result, split_point)

def select_attribute(X, y, method): # OK
    if method == "ID3":
        values, split_points = zip(*[gain(X,y,a,is_continuous[a]) for a in range(X.shape[1])])
        a_star = np.argmax(values)
    elif method == "C4.5":
        gain_values, dumb = zip(*[gain(X,y,a,is_continuous[a]) for a in range(X.shape[1])])
        mask = (gain_values >= np.mean(gain_values)) # the mask is a numpy array! why? because the result of np.mean has type numpy.float64
        gain_ratio_values, split_points = zip(*[gain_ratio(X,y,a,is_continuous[a]) for a in range(X.shape[1])])
        a_star = np.argmax(mask * gain_ratio_values)
    elif method == "CART":
        values, split_points = zip(*[gini_index(X,y,a,is_continuous[a]) for a in range(X.shape[1])])
        a_star = np.argmin(values)
    else:
        raise Exception("Invalid Method")
    return (a_star, split_points[a_star])


class Tree:
    def __init__(self):
        # if this is a leaf, then child == None, kind == kind
        # if this is a branch, then child != None, kind == attribute
        self.child = None 
        self.kind = None
        return
    
    def pprint(self, rev_d_list, label, layer=0):
        if self.child is None:
            print("|  "*layer+"Leaf: "+str(self.kind)+" "+rev_d_list[-1][self.kind])
        elif self.kind[1] is None:
            print("|  "*layer+"Branch: attribute "+str(self.kind[0])+" "+label[self.kind[0]])
            layer=layer+1
            for key, value in self.child.items():
                print("|  "*layer+"Key: "+str(key)+" "+rev_d_list[self.kind[0]][key])
                value.pprint(rev_d_list, label, layer)
        else:
            print("|  "*layer+"Branch: attribute "+str(self.kind[0])+" "+label[self.kind[0]]+" ≤ "+str(self.kind[1]))
            layer=layer+1
            for key, value in self.child.items():
                print("|  "*layer+"Key: "+str(key)+" "+rev_d_list[self.kind[0]][key])
                value.pprint(rev_d_list, label, layer)
                
    def to_mermaid(self, rev_d_list, label):
        
        def mer(node, rev_d_list, label):
            nonlocal node_count
            node_index = node_count
            if node.child is None:
                node_string = "id{}(\"{},{}\")".format(node_index,node.kind,rev_d_list[-1][node.kind])
                string = node_string+"\n"
                node_count = node_count + 1
            elif node.kind[1] is None:
                node_string = "id{}[\"{},{}\"]".format(node_index,node.kind[0],label[node.kind[0]])
                node_count = node_count + 1
                children = ["{} --> |\"{},{}\"| {}".format(node_string,key,rev_d_list[node.kind[0]][key],mer(value, rev_d_list, label)) for key, value in node.child.items()]
                children_string = "\n".join(children)
                string = node_string+"\n"+children_string
            else:
                node_string = "id{}[\"{},{}≤{}\"]".format(node_index,node.kind[0],label[node.kind[0]],node.kind[1])
                node_count = node_count + 1
                children = ["{} --> |\"{},{}\"| {}".format(node_string,key,rev_d_list[node.kind[0]][key],mer(value, rev_d_list, label)) for key, value in node.child.items()]
                children_string = "\n".join(children)
                string = node_string+"\n"+children_string
                
            return string   

        node_count = 0
        string = "graph TB\n"+mer(self,rev_d_list,label)
        string = re.sub('\n+', ';\n', string)
        return string
            
        
    def classify(self, x):
        if self.child is None: # if leaf
            return self.kind
        elif self.kind[1] is None: # if branch, discrete
            return self.child[x[self.kind[0]]].classify(x)
        elif x[self.kind[0]] <= self.kind[1]: # if continuous branch <=
            return self.child[-1].classify(x)
        else:
            return self.child[1].classify(x)
            

    def test(self, X, y):
        result = [self.classify(X[i]) for i in range(len(y))]
        correct_number = [result[i]==y[i] for i in range(len(y))].count(True)
        correct_ratio = correct_number/len(y)
        print("Classify result:")
        print(result)
        print("Real result:")
        print(y)
        print("Total: %d, Correct: %d, Correct ratio: %f"%(len(y), correct_number, correct_ratio))
        return (result, correct_ratio)
    
    def correct_ratio(self, X, y):
        return [self.classify(X[i])==y[i] for i in range(len(y))].count(True)/len(y)
        
    
def generate(X, y, method):
    """X: (n, a)
    y: (a,)
    is_continuous: (a,) type: bool
    method in "ID3", "C4.5", "CART"
    
    global read_only variable: is_continuous
    """

    def gen(X, y, attribute_list, method): # it looks like the attribute_set is never used, so delete it
        node = Tree()
        if np.all(y == y[0]): # if all belongs to one kind
            node.kind = y[0] # assign the tree as a leaf, assign its kind
            return node
        if np.all(X==X[0]): # if no further partition available
            node.kind = y[0]
            node.kind = Counter(y).most_common()[0][0] # take the tree as a leaf, assign the most common kind
            return node

        #select attribute
        pair = select_attribute(X, y, method) # pair: (a_star, split_point)

        node.child = {}
        node.kind = pair # assign attribute
        a_star, split_point = pair
        
        if not is_continuous[a_star]: # discrete
            for attribute in attribute_list[a_star]:
                select_list = (X[:,a_star]==attribute) # the index of the samples with the same attribute
                if not np.any(select_list): # if no sample has the attribute
                    node.child[attribute] = Tree()
                    node.child[attribute].kind = Counter(y).most_common()[0][0] # take the tree as a leaf, assign the most common kind
                else:
                    node.child[attribute] = gen(X[select_list], y[select_list], attribute_list, method) # recursively construct the tree
            return node
        else: # continuous, there must be a split_point if a continuous attribute was selected
            assert split_point != None
            select_list = (X[:,a_star]<=split_point)
            node.child[-1] = gen(X[select_list], y[select_list], attribute_list, method) # recursively construct the neg tree
            select_list = (X[:,a_star]>split_point)
            node.child[1] = gen(X[select_list], y[select_list], attribute_list, method) # recursively construct the pos tree
            return node

    attribute_list = [np.unique(attributes) for attributes in X.T]
    tree = gen(X, y, attribute_list, method)
    return tree
    

def generate_with_pre_pruning(X, y, X_val, y_val, method):

    def gen(X, y, X_val, y_val, attribute_list, method):
        node = Tree()
        if np.all(y == y[0]): # if all belongs to one kind
            node.kind = y[0] # assign the tree as a leaf, assign its kind
            return node
        if np.all(X==X[0]): # if no further partition available
            node.kind = y[0]
            node.kind = Counter(y).most_common()[0][0] # take the tree as a leaf, assign the most common kind
            return node
        
        #select attribute
        pair = select_attribute(X, y, method) # pair: (a_star, split_point)
            
        #pre-pruning

        # assume this is a branch, calculate correct ratio
        node.child = {}
        node.kind = pair # assign attribute
        a_star, split_point = pair

        if not is_continuous[a_star]: # discrete
            for attribute in attribute_list[a_star]:
                select_list = (X[:,a_star]==attribute) # the index of the samples with the same attribute
                node.child[attribute] = Tree()
                if np.any(select_list):
                    node.child[attribute].kind = Counter(y[select_list]).most_common()[0][0] # take the tree as a leaf, assign the most common kind
                else:
                    node.child[attribute].kind = Counter(y).most_common()[0][0] # take the tree as a leaf, assign the most common kind
        else:
            assert split_point != None
            select_list = (X[:,a_star]<=split_point) # select small value
            node.child[-1] = Tree()
            if np.any(select_list):
                node.child[-1].kind = Counter(y[select_list]).most_common()[0][0] # take the tree as a leaf, assign the most common kind
            else:
                node.child[-1].kind = Counter(y).most_common()[0][0] # take the tree as a leaf, assign the most common kind
            select_list = (X[:,a_star]>split_point) # reassign select_list, select larger value
            node.child[1] = Tree()
            if np.any(select_list):
                node.child[1].kind = Counter(y[select_list]).most_common()[0][0] # take the tree as a leaf, assign the most common kind
            else:
                node.child[1].kind = Counter(y).most_common()[0][0] # take the tree as a leaf, assign the most common kind
        correct_ratio_as_branch = node.correct_ratio(X_val, y_val) # calc correct ratio
            
        # otherwise, assume this is a leaf, calculate correct ratio
        node.child = None
        node.kind = Counter(y).most_common()[0][0] # take the tree as a leaf, assign the most common kind
        correct_ratio_as_leaf = node.correct_ratio(X_val, y_val) # calc correct ratio
        if correct_ratio_as_leaf >= correct_ratio_as_branch: # trim
            return node
                
        # if untrimmed, restore node child and kind
        node.child = {}
        node.kind = pair # assign attribute
        a_star, split_point = pair
        
        if not is_continuous[a_star]: # discrete
            for attribute in attribute_list[a_star]:
                select_list = (X[:,a_star]==attribute) # the index of the samples with the same attribute
                val_list = (X_val[:,a_star]==attribute) # the index of the validation samples with the same attribute
                if not np.any(select_list): # if no sample has the attribute
                    node.child[attribute] = Tree()
                    node.child[attribute].kind = Counter(y).most_common()[0][0] # take the tree as a leaf, assign the most common kind
                else:
                    node.child[attribute] = gen(X[select_list], y[select_list], X_val[val_list], y_val[val_list], attribute_list, method) # recursively construct the tree
            return node
        else: # continuous, there must be a split_point if a continuous attribute was selected
            assert split_point != None
            select_list = (X[:,a_star]<=split_point)
            val_list = (X_val[:,a_star]<=attribute)
            node.child[-1] = gen(X[select_list], y[select_list], X_val[val_list], y_val[val_list], attribute_list, method) # recursively construct the neg tree
            select_list = (X[:,a_star]>split_point)
            val_list = (X_val[:,a_star]>attribute)
            node.child[1] = gen(X[select_list], y[select_list], X_val[val_list], y_val[val_list], attribute_list, method) # recursively construct the pos tree
            return node

    attribute_list = [np.unique(attributes) for attributes in X.T]
    tree = gen(X, y, X_val, y_val, attribute_list, method)
    return tree
        
    
def generate_with_post_pruning(X, y, X_val, y_val, method):

    def gen(X, y, attribute_list, method): # it looks like the attribute_set is never used, so delete it
        node = Tree()
        if np.all(y == y[0]): # if all belongs to one kind
            node.kind = y[0] # assign the tree as a leaf, assign its kind
            return node
        if np.all(X==X[0]): # if no further partition available
            node.kind = y[0]
            node.kind = Counter(y).most_common()[0][0] # take the tree as a leaf, assign the most common kind
            return node
        
        #if node is not leaf, push node
        node_list.append(node)
        
        #select attribute
        pair = select_attribute(X, y, method) # pair: (a_star, split_point)

        node.child = {}
        node.kind = pair # assign attribute
        a_star, split_point = pair
        
        if not is_continuous[a_star]: # discrete
            for attribute in attribute_list[a_star]:
                select_list = (X[:,a_star]==attribute) # the index of the samples with the same attribute
                if not np.any(select_list): # if no sample has the attribute
                    node.child[attribute] = Tree()
                    node.child[attribute].kind = Counter(y).most_common()[0][0] # take the tree as a leaf, assign the most common kind
                else:
                    node.child[attribute] = gen(X[select_list], y[select_list], attribute_list, method) # recursively construct the tree
            return node
        else: # continuous, there must be a split_point if a continuous attribute was selected
            assert split_point != None
            select_list = (X[:,a_star]<=split_point)
            node.child[-1] = gen(X[select_list], y[select_list], attribute_list, method) # recursively construct the neg tree
            select_list = (X[:,a_star]>split_point)
            node.child[1] = gen(X[select_list], y[select_list], attribute_list, method) # recursively construct the pos tree
            return node
    
    def post_pruning(tree, X_val, y_val):
        while len(node_list) > 0:
            # pop out the newest node
            node = node_list.pop() # now this node is still attached to the tree!
            # assume this node is a branch
            correct_ratio_as_branch = tree.correct_ratio(X_val, y_val)
            # now assume this node is a leaf
            child = node.child # store node.child
            kind = node.kind # store node.kind
            node.child = None
            node.kind = Counter(y).most_common()[0][0] # take the tree as a leaf, assign the most common kind
            correct_ratio_as_leaf = tree.correct_ratio(X_val, y_val) # calc correct ratio
            if correct_ratio_as_leaf <= correct_ratio_as_branch: # if not trimming, then restore node.child and node.kind (in order to get the same result as shown in the textbook, change < to <=, see P82 left margin)
                node.child = child
                node.kind = kind
                
    node_list = []
    attribute_list = [np.unique(attributes) for attributes in X.T]
    tree = gen(X, y, attribute_list, method)
    post_pruning(tree, X_val, y_val)
    return tree

        

## Run

In [20]:


t = time()
dataSet1 = dataSet.to_numpy()
X = dataSet1[:,:-1]
X0 = X[:,:-2]
y = dataSet1[:,-1]

# reproduce fig 4.4
tree = generate(X0, y, "ID3")
tree.pprint(rev_d_list, label)
tree.test(X0, y)
print(tree.to_mermaid(rev_d_list, label))
print()

shuffle = [4,0,1,2,3,5,8]
X1 = X[:,shuffle[:-1]]
# rev_d_list = re[shuffle]
test_selection = np.array([1,2,3,6,7,10,14,15,16,17])-1
val_selection = np.array([4,5,8,9,11,12,13])-1
X_test = X1[test_selection,:]
y_test = y[test_selection]
X_val = X1[val_selection,:]
y_val = y[val_selection]
            
# reproduce fig 4.5
tree2 = generate(X_test, y_test, "ID3")
tree2.pprint([rev_d_list[i] for i in shuffle], [label[i] for i in shuffle])
tree2.test(X_val, y_val)
print(tree2.to_mermaid([rev_d_list[i] for i in shuffle], [label[i] for i in shuffle]))
print()

# reproduce fig 4.6
tree3 = generate_with_pre_pruning(X_test, y_test, X_val, y_val, "ID3")
tree3.pprint([rev_d_list[i] for i in shuffle], [label[i] for i in shuffle])
tree3.test(X_val, y_val)
print(tree3.to_mermaid([rev_d_list[i] for i in shuffle], [label[i] for i in shuffle]))
print()

# when change the "<" to "<=" in post_pruning(tree, X_val, y_val)), reproduce fig 4.7
tree4 = generate_with_post_pruning(X_test, y_test, X_val, y_val, "ID3")
tree4.pprint([rev_d_list[i] for i in shuffle], [label[i] for i in shuffle])
tree4.test(X_val, y_val)
print(tree4.to_mermaid([rev_d_list[i] for i in shuffle], [label[i] for i in shuffle]))
print()

# reproduce fig 4.8
tree5 = generate(X, y, "ID3")
tree5.pprint(rev_d_list, label)
tree5.test(X, y)
print(tree5.to_mermaid(rev_d_list, label))
print()


time() - t

Branch: attribute 3 纹理
|  Key: -1.0 模糊
|  Leaf: -1.0 坏瓜
|  Key: 0.0 稍糊
|  Branch: attribute 5 触感
|  |  Key: -1.0 软粘
|  |  Leaf: 1.0 好瓜
|  |  Key: 1.0 硬滑
|  |  Leaf: -1.0 坏瓜
|  Key: 1.0 清晰
|  Branch: attribute 1 根蒂
|  |  Key: -1.0 硬挺
|  |  Leaf: -1.0 坏瓜
|  |  Key: 0.0 稍蜷
|  |  Branch: attribute 0 色泽
|  |  |  Key: -1.0 浅白
|  |  |  Leaf: 1.0 好瓜
|  |  |  Key: 0.0 青绿
|  |  |  Leaf: 1.0 好瓜
|  |  |  Key: 1.0 乌黑
|  |  |  Branch: attribute 5 触感
|  |  |  |  Key: -1.0 软粘
|  |  |  |  Leaf: -1.0 坏瓜
|  |  |  |  Key: 1.0 硬滑
|  |  |  |  Leaf: 1.0 好瓜
|  |  Key: 1.0 蜷缩
|  |  Leaf: 1.0 好瓜
Classify result:
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]
Real result:
[ 1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
Total: 17, Correct: 17, Correct ratio: 1.000000
graph TB;
id0["3,纹理"];
id0["3,纹理"] --> |"-1.0,模糊"| id1("-1.0,坏瓜");
id0["3,纹理"] --> |"0.0,稍糊"| id2["5,触感"];
id2["5,触感"] --> |"-1.0,软粘"| id3("1.0,好瓜");
id2["5,触感"] --> |"1.0,硬滑"| id4

0.05052900314331055